# Imports

In [1]:
import pandas as pd

In [27]:
import nltk
import re

from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mary\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Mary\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mary\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Mary\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
import numpy as np
# def dummy_npwarn_decorator_factory():
#   def npwarn_decorator(x):
#     return x
#   return npwarn_decorator
# np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)

In [28]:
from bertopic import BERTopic

In [ ]:
from transformers import pipeline #if error change version of transformers

c:\Users\mkavalio\codedir\TextAnalysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


# Dataset import and text preprocessing

In [72]:
df = pd.read_csv(r"D:\workdir\TopicModelling\Data\Car_Reviews_Database_KaggleSentiment.csv", encoding='ISO-8859-1')
df = df.dropna()
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 234 entries, 0 to 277
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Year    234 non-null    int64 
 1   Model   234 non-null    object
 2   Review  234 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.3+ KB


In [71]:
stop_words = set(stopwords.words('english'))
add_to_stop = ["car", "buick", "cadillac", "chevrolet", "chrysler", "dodge", "ford", "gmc", "honda", "hyundai", "jeep", "kia", "mazda", \
               "nissan", "subaru", "toyota", "volkswagen", "volvo"]
for word in add_to_stop:
    stop_words.add(word)
print(stop_words)

def preprocess_text(text: str, stopwords=stop_words) -> str:
    # remove links
    text = re.sub(r"http\S+", "", text)
    # remove special chars and numbers
    text = re.sub("[^A-Za-z]+", " ", text) # leave only abc characters and spaces
    
    # remove stopwords
  
    tokens = nltk.word_tokenize(text)
    tokens = [w for w in tokens if w.lower() not in stopwords]
    text = " ".join(tokens)
    text = text.lower().strip() # removes leading and trailing whitespaces
    
    return text

{'ford', 'by', 'hyundai', 'because', 'it', 'below', 'both', "it's", 'her', 'you', 'y', 'through', "hadn't", "that'll", 'off', 'only', 'hers', 'themselves', 'they', 'am', 'himself', 'its', 'yourself', 'these', 'each', 'cadillac', 'over', 'i', 'ourselves', 'our', 'out', "don't", 'of', 'not', 'does', 'did', 'car', 'the', 'being', 'haven', 'mustn', "you'll", 'are', 'then', 'to', 'she', 'about', 'most', 'yours', 'when', "wouldn't", 'here', 'll', 'at', "should've", 'other', 'more', 'between', 'any', 'do', 'down', 'there', 'yourselves', "needn't", 'with', "aren't", 'same', 'just', 'on', 'for', 'theirs', 'my', "weren't", 'so', "you're", 'from', 'chevrolet', 'while', 'can', 'now', 'mightn', 'your', 't', "shan't", 'couldn', 'he', 'that', 'few', 'too', 'needn', 'wasn', "she's", 'dodge', 'was', 'their', 'were', "didn't", 'such', 'no', 'honda', 'toyota', 'wouldn', 'kia', 'him', 'above', 'very', "wasn't", 'herself', 'own', 'if', 'had', 'after', 'before', 'against', 'shouldn', 'will', 'we', 'all', 'u

In [73]:
df['Review_processed'] = df['Review'].apply(lambda text: preprocess_text(text))
docs = df['Review_processed'].tolist()
df.sample(2)

,Year,Model,Review,Review_processed
60,2018,GMC,Key upgrade is the 2018 Yukon Denali becomes o...,key upgrade yukon denali becomes one first suv...
36,2016,Hyundai,Enter the 2016 Hyundai Elantra Value Edition t...,enter elantra value edition offers enough give...


# Experiments

## BERTopic

### Basics

In [67]:
topic_model = BERTopic()


In [74]:
topics, probs = topic_model.fit_transform(docs)

In [75]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,44,-1_model_also_luxurious_world,"[model, also, luxurious, world, models, mulsan...",[recently seen super luxurious model coming br...
1,0,149,0_drive_like_driving_great,"[drive, like, driving, great, good, even, spee...",[vehicle long stood excess opulence knew going...
2,1,22,1_handling_engine_interior_camaro,"[handling, engine, interior, camaro, ranks, cl...",[despite athletic handling upscale cabin regal...
3,2,19,2_bmw_series_handling_luxury,"[bmw, series, handling, luxury, rear, small, s...",[bmw series muscular engines nimble handling a...


In [77]:
topic_model.get_topic(0)

[('drive', np.float64(0.028464994770944746)),
 ('like', np.float64(0.02826070948020483)),
 ('driving', np.float64(0.02776705583973617)),
 ('great', np.float64(0.02667612446943073)),
 ('good', np.float64(0.02641275855629422)),
 ('even', np.float64(0.023116349820506067)),
 ('speed', np.float64(0.021511383988461412)),
 ('vehicle', np.float64(0.02138206730751355)),
 ('one', np.float64(0.02062697612355299)),
 ('well', np.float64(0.02038947237905064))]

In [ ]:
topic_model.get_document_info(docs)

### Fine-tune Topic Representations

In [84]:
from bertopic.representation import KeyBERTInspired

In [85]:
representation_model = KeyBERTInspired()
topic_model = BERTopic(representation_model=representation_model)

In [88]:
topics, probs = topic_model.fit_transform(docs)

In [89]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,56,-1_luxurious_luxury_richest_bentley,"[luxurious, luxury, richest, bentley, cars, mo...",[recently seen super luxurious model coming br...
1,0,131,0_audi_vehicle_cars_driving,"[audi, vehicle, cars, driving, suv, highway, e...",[vehicle long stood excess opulence knew going...
2,1,27,1_automotive_regal_seats_luxurious,"[automotive, regal, seats, luxurious, sedan, c...",[despite athletic handling upscale cabin regal...
3,2,20,2_bmw_mercedes_sedan_luxury,"[bmw, mercedes, sedan, luxury, audi, luxurious...",[bmw series muscular engines nimble handling a...
